In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#from google.colab import files
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset/creditcard.csv')

In [8]:
import pandas as pd
import numpy as np
import datetime as datetime
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import SimpleRNN, Input, Embedding, Dense, LSTM, LeakyReLU, LayerNormalization, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential, Model

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [11]:
df_ = pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset/ransomware.csv')

In [10]:
df_.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [ ]:
tmp = df_.loc[df_['SrcAddr']=='192.168.122.100']
y = np.array(tmp['Label'])
X = tmp.drop(['StartTime','SrcAddr','TotPkts','Label','datatype','sTos','SrcPkts'
             ], axis=1).values

In [ ]:
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, random_state=100, shuffle = False)

In [ ]:
scaler = MinMaxScaler().fit(trainX)
trainX_rescaled = scaler.transform(trainX)
testX_rescaled = scaler.transform(testX)

In [ ]:
trainX_rescaled.shape

In [ ]:
%%time
# Create overlapping windows of lagged values for training and testing datasets
timesteps = 50 
train_generator = TimeseriesGenerator(trainX_rescaled, trainY, length = timesteps, sampling_rate = 1)#, batch_size = timesteps)
test_generator = TimeseriesGenerator(testX_rescaled, testY, length = timesteps, sampling_rate = 1)#, batch_size = timesteps)

train_X, train_y = train_generator[0]
test_X, test_y = test_generator[0]

print(timesteps, trainX.shape, test_X.shape)

In [ ]:
#help(train_generator)
x_ = next(iter(train_generator))[0]
y_ = next(iter(train_generator))[1]

print(x_.shape, y_.shape)
print(x_[0])

In [ ]:
input_data = Input(shape=(50,54))
#embed_out = Embedding(vocab_size, 64, input_length=max_len)(embed_input)
X = Bidirectional(SimpleRNN(50, return_sequences=True))(input_data)
X = LayerNormalization()(X)
X = LeakyReLU()(X)
X = LSTM(32)(X)
X = Dense(1, activation='sigmoid')(X)
rnn_model = Model(input_data, X)

In [ ]:
rnn_model.summary()

In [ ]:
units = 50 # 출력크기
num_epoch = 5
learning_rate = 0.001

In [ ]:
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = rnn_model.fit_generator(train_generator,\
                        epochs = num_epoch, \
                        validation_data = test_generator, \
                        #callbacks = [tensorboard_callback], \
                        verbose= 1, \
                        shuffle=False)

In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
ypred = rnn_model.predict(test_generator)

In [ ]:
ypred_= np.where(ypred>0.5,1,0)

import sklearn.metrics as metrics
print(metrics.confusion_matrix(ypred_,testY[50:]))
print(metrics.accuracy_score(testY[50:],ypred_))
print(metrics.f1_score(testY[50:],ypred_))


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(testY[50:], ypred)
roc_auc = metrics.auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()